# Model for predicting Second Language Adquisiton

We start by loading the necessary libraries.

In [1]:
from fastai.data.all import *
from fastai.learner import *
from fastai.losses import CrossEntropyLossFlat
from fastai.optimizer import Adam, OptimWrapper, params
from fastai.metrics import accuracy, F1Score
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForSequenceClassification
import pandas as pd
from blurr.text.data.core import *
from blurr.text.modeling.core import *
from blurr.text.utils import NLP as BLURR # lo importamos así para que sea compatible con el resto
from datasets import load_dataset, concatenate_datasets

/home/jonathan/.virtualenvs/huggingface/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import torch
torch.cuda.set_device(0)

Now, we load the train and validation sets.

In [3]:
train_df = pd.read_csv('perplejidad/datasets/train_train_set.csv',delimiter='\t')

In [4]:
valid_df = pd.read_csv('perplejidad/datasets/validation_train_set.csv',delimiter='\t')

In [5]:
len(train_df),len(valid_df)

(33684, 3743)

In [6]:
train_df['is_valid']=False

In [7]:
valid_df['is_valid']=True

In [8]:
train_valid_df = pd.concat([train_df,valid_df])

We define the indexes for training and validation.

In [9]:
train_idxs, valid_idxs = L(range(len(train_valid_df[train_valid_df.is_valid==False]))), L(range(len(train_valid_df[train_valid_df.is_valid==False]), len(train_valid_df)))

Now, we can define the model. In this case, we use a predefined model called Roberta.

In [10]:
dl_kwargs = {"bs": 64, "val_bs": 64}
learn_kwargs = {"metrics": [accuracy]}

In [11]:
learn = BlearnerForSequenceClassification.from_data(
    train_valid_df,
    "xlm-roberta-base",
    text_attr="Sentence",
    label_attr="Level",
    dblock_splitter=IndexSplitter(valid_idxs),
    dl_kwargs=dl_kwargs,
    learner_kwargs=learn_kwargs,
)

In [12]:
learn = learn.to_fp16()

In [13]:
learn.load('xlm-roberta-base')

In [15]:
df = pd.read_csv('sentences40RandomShuffle.txt',sep='\t')

In [16]:
df

,Unnamed: 0,Number,Word,Sentence,PpW2,PpW3,PpW4,PpW5,PpKenlm,Mother tongue,Level
0,53956.0,38290,.,No recuerdo quién fue director+(a) director+(a) .,4935.440739,7894.825367,9859.754361,10046.592115,33261.523580,Inglés,C1
1,NaN,634419,voy,Apenas para avisar que voy a llegar tarde hoy porque después de la clase voy con mis amigas a casa de Maria .,5390.592395,13624.724596,24718.453222,31443.724124,507.470311,Portugués,A1
2,NaN,492435,lugares,"Pienso que fumar debe ser prohibido , especialmente en lugares donde hay mucha gente .",4295.478049,9129.695596,14100.635694,16184.557753,2156.641234,Ruso,B2
3,61692.0,113485,de,"Mis actividades preferidas son nadar , eskiar , leer , y el ayudo informatico de cualquier ser humano .",11320.079084,20287.227241,27890.224591,29868.151413,41580.820144,Árabe,B2
4,25922.0,143663,el,Viven en otros estados de el País .,645.436794,1394.222521,2168.832940,2579.562930,4631.701291,Portugués,A1
5,NaN,600711,teléfono,"O también puedes llamar me por mi teléfono , 0987619777 .",14610.122711,23504.157036,22589.963576,18524.310656,15489.398333,Chino mandarín,B1
6,NaN,202448,me,"Si tenían prisa me llmarán por telefono , por_favor .",53961.034908,47057.106722,33576.796312,23206.521060,40171.582693,Chino mandarín,A1
7,NaN,501566,menos,te hecho de menos te salodo ibrahim,32956.239335,36281.501084,26751.017175,18447.824972,2655.190164,Árabe,B1
8,84915.0,453949,gustaría,Me gustaría saber,27694.309283,14619.256805,8138.489608,4916.534359,4796.604961,Ruso,B1
9,53400.0,583724,Señores,Estimdos Señores,184295.210598,32707.777236,11085.199147,5329.540449,19890.861921,Árabe,C1


In [18]:
tfm = first_blurr_tfm(learn.dls)
hf_tokenizer = tfm.hf_tokenizer
hf_tokenizer.unk_token

'<unk>'

In [20]:
class_labels = ['A1', 'A2', 'B1', 'B2', 'C1']

sentence = 0

pred = learn.predict(df['Sentence'][sentence])[0]
print(df['Sentence'][sentence],class_labels[pred['class_index']],pred['score'])

words = df['Sentence'][sentence].split(' ')
res = []
for i in range(0,len(words)):
    transform = df['Sentence'][sentence].split(' ')
    transform[i] = hf_tokenizer.unk_token
    pred = learn.predict(' '.join(transform))[0]
    res.append([' '.join(transform) , class_labels[pred['class_index']],str(pred['score'])])
    
pd.DataFrame(res)
    




No recuerdo quién fue director+(a) director+(a) . C1 0.9571301341056824


,0,1,2
0,<unk> recuerdo quién fue director+(a) director+(a) .,C1,0.7719404101371765
1,No <unk> quién fue director+(a) director+(a) .,C1,0.5735546350479126
2,No recuerdo <unk> fue director+(a) director+(a) .,A2,0.99471515417099
3,No recuerdo quién <unk> director+(a) director+(a) .,C1,0.9996607303619385
4,No recuerdo quién fue <unk> director+(a) .,C1,0.9992815852165222
5,No recuerdo quién fue director+(a) <unk> .,C1,0.9972475171089172
6,No recuerdo quién fue director+(a) director+(a) <unk>,C1,0.865392804145813


In [65]:
class_labels = ['A1', 'A2', 'B1', 'B2', 'C1']
    
df['prediction'] = df['Sentence'].apply(lambda x : class_labels[learn.predict(x)[0]['class_index']])

In [66]:
df

,Unnamed: 0,Number,Word,Sentence,PpW2,PpW3,PpW4,PpW5,PpKenlm,Mother tongue,Level,prediction
0,53956.0,38290,.,No recuerdo quién fue director+(a) director+(a) .,4935.440739,7894.825367,9859.754361,10046.592115,33261.523580,Inglés,C1,C1
1,NaN,634419,voy,Apenas para avisar que voy a llegar tarde hoy porque después de la clase voy con mis amigas a casa de Maria .,5390.592395,13624.724596,24718.453222,31443.724124,507.470311,Portugués,A1,A1
2,NaN,492435,lugares,"Pienso que fumar debe ser prohibido , especialmente en lugares donde hay mucha gente .",4295.478049,9129.695596,14100.635694,16184.557753,2156.641234,Ruso,B2,B2
3,61692.0,113485,de,"Mis actividades preferidas son nadar , eskiar , leer , y el ayudo informatico de cualquier ser humano .",11320.079084,20287.227241,27890.224591,29868.151413,41580.820144,Árabe,B2,A1
4,25922.0,143663,el,Viven en otros estados de el País .,645.436794,1394.222521,2168.832940,2579.562930,4631.701291,Portugués,A1,A1
5,NaN,600711,teléfono,"O también puedes llamar me por mi teléfono , 0987619777 .",14610.122711,23504.157036,22589.963576,18524.310656,15489.398333,Chino mandarín,B1,B1
6,NaN,202448,me,"Si tenían prisa me llmarán por telefono , por_favor .",53961.034908,47057.106722,33576.796312,23206.521060,40171.582693,Chino mandarín,A1,B1
7,NaN,501566,menos,te hecho de menos te salodo ibrahim,32956.239335,36281.501084,26751.017175,18447.824972,2655.190164,Árabe,B1,B1
8,84915.0,453949,gustaría,Me gustaría saber,27694.309283,14619.256805,8138.489608,4916.534359,4796.604961,Ruso,B1,B2
9,53400.0,583724,Señores,Estimdos Señores,184295.210598,32707.777236,11085.199147,5329.540449,19890.861921,Árabe,C1,B2
